**1.读取数据**

In [47]:
from sklearn.datasets import fetch_20newsgroups


# 即时从网上下载数据
news=fetch_20newsgroups(subset='all')

print len(news.data)
print news.data[0]

18846
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




数据没有被设定特征，也无数字化的量度，因此需要在被训练前对数据做进一步处理。

**2.分割数据**

In [48]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(news.data,news.target,test_size=0.25,random_state=33)

**3.使用朴素贝叶斯分类器对新闻文本数据进行类别预测**

- 先将文本转化为特征向量
- 再利用朴素贝叶斯模型从训练数据中估计参数
- 最后利用这些概率参数对同样转化为特征向量的测试集进行类别预测

In [44]:
# 导入用于文本特征向量转换模块
from sklearn.feature_extraction.text import CountVectorizer
vec=CountVectorizer()
X_train=vec.fit_transform(X_train)
X_test=vec.transform(X_test)

# 导入naive bayes
from sklearn.naive_bayes import MultinomialNB
# 初始化NB
mnb=MultinomialNB()
# 利用训练数据对模型参数进行估计
mnb.fit(X_train,y_train)
# 对测试样本进行类别预测，结果存储在变量y_predict中
y_predict=mnb.predict(X_test)

**4.性能评估**

In [45]:
from sklearn.metrics import classification_report
print 'The Accuracy of NBC is:',mnb.score(X_test,y_test)
print classification_report(y_test,y_predict,target_names=news.target_names)

由上评估结果可知，NBC对4712条新闻文本测试样本分类的准确性约为83.977%，平均精确率、召回率、F1指标分别为86%、84%、82%。

特点分析：

- 朴素贝叶斯模型广泛应用在互联网文本分类任务
- 优点：由于其较强的特征条件独立假设，使得模型预测所需估计的参数规模从幂指数量级向线性量级减少，极大节约内存消耗和计算时间
- 缺点：同样由于这种强假设的限制，模型训练时无法将各个特征之间的联系考量在内，使该模型在其他数据特征关联性强的分类任务上性能不佳

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec=TfidfVectorizer()

# 转化为特征向量
X_tfidf_train=tfidf_vec.fit_transform(X_train)
X_tfidf_test=tfidf_vec.transform(X_test)

from sklearn.naive_bayes import MultinomialNB
mnb_tfidf=MultinomialNB()
mnb_tfidf.fit(X_tfidf_train,y_train)
y_tfidf_predict=mnb_tfidf.predict(X_tfidf_test)

# 性能评估
print 'Accuracy:',mnb_tfidf.score(X_tfidf_test,y_test)
from sklearn.metrics import classification_report
print classification_report(y_test,y_tfidf_predict,target_names=news.target_names)

Accuracy: 0.8463497453310697
                          precision    recall  f1-score   support

             alt.atheism       0.84      0.67      0.75       201
           comp.graphics       0.85      0.74      0.79       250
 comp.os.ms-windows.misc       0.82      0.85      0.83       248
comp.sys.ibm.pc.hardware       0.76      0.88      0.82       240
   comp.sys.mac.hardware       0.94      0.84      0.89       242
          comp.windows.x       0.96      0.84      0.89       263
            misc.forsale       0.93      0.69      0.79       257
               rec.autos       0.84      0.92      0.88       238
         rec.motorcycles       0.98      0.92      0.95       276
      rec.sport.baseball       0.96      0.91      0.94       251
        rec.sport.hockey       0.88      0.99      0.93       233
               sci.crypt       0.73      0.98      0.83       238
         sci.electronics       0.91      0.83      0.87       249
                 sci.med       0.97      0.92 

由输出可知，在使用TfidfVectorizer而不去掉停用词的条件下，对训练和测试文本进行特征量化，并利用默认配置的naive bayes，在测试文本上可得到比CountVectorizer更高的预测准确性。证明：在训练文本量较多时，使用TfidfVectorizer压制常用词汇对分类决策的干扰，可提升模型性能。

**4.分别使用CountVectorizer和TfidfVectorizer，并在去掉停用词的条件下，对文本特征进行量化的Naive Bayes分类性能测试**

In [49]:
# 分别使用停用词过滤器配置初始化CountVectorizer和TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
count_filter_vec,tfidf_filter_vec=CountVectorizer(analyzer='word',stop_words='english'),TfidfVectorizer(analyzer='word',stop_words='english')

# 使用带停用词过滤的CountVectorizer对训练和测试文本进行量化处理
X_count_filter_train=count_filter_vec.fit_transform(X_train)
X_count_filter_test=count_filter_vec.transform(X_test)

# 使用带停用词过滤的TfidfVectorizer对训练和测试文本进行量化处理
X_tfidf_filter_train=tfidf_filter_vec.fit_transform(X_train)
X_tfidf_filter_test=tfidf_filter_vec.transform(X_test)


# 初始化默认配置的朴素贝叶斯，并对CountVectorizer后的数据进行预测和性能评估
from sklearn.naive_bayes import MultinomialNB
mnb_count_filter=MultinomialNB()
mnb_count_filter.fit(X_count_filter_train,y_train)
y_count_predict=mnb_count_filter.predict(X_count_filter_test)

# 初始化另一个默认配置的朴素贝叶斯，并对TfidfVectorizer后的数据进行预测和性能评估
mnb_tfidf_filter=MultinomialNB()
mnb_tfidf_filter.fit(X_tfidf_filter_train,y_train)
y_tfidf_predict=mnb_tfidf_filter.predict(X_tfidf_filter_test)

# CountVectorizer性能评估
from sklearn.metrics import classification_report
print 'Count_Accuracy',mnb_count_filter.score(X_count_filter_train,y_train)
print classification_report(y_test,y_count_predict,target_names=news.target_names)

# TfidfVectorizer性能评估
print 'Tfidf_Accuracy',mnb_tfidf_filter.score(X_tfidf_filter_train,y_train)
print classification_report(y_test,y_tfidf_predict,target_names=news.target_names)






Count_Accuracy 0.9439649073156926
                          precision    recall  f1-score   support

             alt.atheism       0.85      0.89      0.87       201
           comp.graphics       0.62      0.88      0.73       250
 comp.os.ms-windows.misc       0.93      0.22      0.36       248
comp.sys.ibm.pc.hardware       0.62      0.88      0.73       240
   comp.sys.mac.hardware       0.93      0.85      0.89       242
          comp.windows.x       0.82      0.85      0.84       263
            misc.forsale       0.90      0.79      0.84       257
               rec.autos       0.91      0.91      0.91       238
         rec.motorcycles       0.98      0.94      0.96       276
      rec.sport.baseball       0.98      0.92      0.95       251
        rec.sport.hockey       0.92      0.99      0.95       233
               sci.crypt       0.91      0.97      0.93       238
         sci.electronics       0.87      0.89      0.88       249
                 sci.med       0.94      

由输出可知，TfidfVectorizer的特征抽取和量化方法更具备优势，对停用词进行过滤后的模型性能比未过滤高3%—4%。